In [114]:
#importando as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [115]:

df = pd.read_csv("all_bovespa.csv", delimiter=',')
df

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,volume_negocios
0,2017-01-02,AALR3,ALLIAR,14.62,14.88,14.40,14.60,35900,52350500
1,2017-01-02,ABCB4,ABC BRASIL,13.40,13.52,13.08,13.31,195800,260557000
2,2017-01-02,ABEV3,AMBEV S/A,16.34,16.66,16.26,16.31,4265400,7004006700
3,2017-01-02,ADHM3,ADVANCED-DH,0.81,0.83,0.76,0.79,68600,5407500
4,2017-01-02,AFLT3,AFLUENTE T,3.18,3.18,3.18,3.18,100,31800
...,...,...,...,...,...,...,...,...,...
577768,2021-12-27,V1TA34,VENTAS INC,139.02,139.72,139.02,139.72,60,865420
577769,2021-12-28,V1TA34,VENTAS INC,140.86,140.86,140.68,140.68,0,126700
577770,2021-12-28,MYPK3,IOCHP-MAXION,15.02,15.32,14.91,15.22,836300,1263298800
577771,2021-12-29,MYPK3,IOCHP-MAXION,15.21,15.48,14.90,14.90,776000,1171092600


In [116]:
df_itau = df[df['sigla_acao'] == 'ITUB4' ]

In [117]:
#Mudar o tipo data
df_itau['data_pregao'] = pd.to_datetime(df_itau['data_pregao'], format='%Y-%m-%d')
df_itau.dtypes

C:\Users\mille\AppData\Local\Temp\ipykernel_14520\422081031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itau['data_pregao'] = pd.to_datetime(df_itau['data_pregao'], format='%Y-%m-%d')


data_pregao         datetime64[ns]
sigla_acao                  object
nome_acao                   object
preco_abertura             float64
preco_max                  float64
preco_min                  float64
preco_fechamento           float64
qtd_negocios                 int64
volume_negocios              int64
dtype: object

In [118]:
#criando novos campos de medias móveis
df_itau['mm5d'] = df_itau['preco_fechamento'].rolling(5).mean()
df_itau['mm21d'] = df_itau['preco_fechamento'].rolling(21).mean()

C:\Users\mille\AppData\Local\Temp\ipykernel_14520\1600553900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itau['mm5d'] = df_itau['preco_fechamento'].rolling(5).mean()
C:\Users\mille\AppData\Local\Temp\ipykernel_14520\1600553900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itau['mm21d'] = df_itau['preco_fechamento'].rolling(21).mean()


In [119]:
#Empurrando para frente os valores das ações
df_itau['preco_fechamento'] = df_itau['preco_fechamento'].shift(-1)

df_itau.head()

C:\Users\mille\AppData\Local\Temp\ipykernel_14520\2053566568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itau['preco_fechamento'] = df_itau['preco_fechamento'].shift(-1)


,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,volume_negocios,mm5d,mm21d
137,2017-01-02,ITUB4,ITAUUNIBANCO,33.45,33.74,33.31,34.95,2070100,6939867600,NaN,NaN
436,2017-01-03,ITUB4,ITAUUNIBANCO,33.84,34.97,33.78,35.10,16810000,58389512200,NaN,NaN
755,2017-01-04,ITUB4,ITAUUNIBANCO,34.80,35.23,34.71,35.58,9079500,31781554500,NaN,NaN
1072,2017-01-05,ITUB4,ITAUUNIBANCO,35.09,35.98,34.81,35.70,14659200,52130994300,NaN,NaN
1370,2017-01-06,ITUB4,ITAUUNIBANCO,35.54,35.87,35.21,35.87,9345100,33299308400,34.972,NaN


In [120]:
#retirando os dados nulos
df_itau.dropna(inplace=True)
df_itau

C:\Users\mille\AppData\Local\Temp\ipykernel_14520\2300908495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itau.dropna(inplace=True)


,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_max,preco_min,preco_fechamento,qtd_negocios,volume_negocios,mm5d,mm21d
6457,2017-01-31,ITUB4,ITAUUNIBANCO,37.26,37.47,36.84,37.62,11259900,41891709700,37.704,36.387143
6791,2017-02-01,ITUB4,ITAUUNIBANCO,37.55,37.85,37.41,37.51,13739900,51731266500,37.680,36.581905
7118,2017-02-02,ITUB4,ITAUUNIBANCO,37.28,37.83,36.61,38.29,14397000,53952339800,37.532,36.703810
7474,2017-02-03,ITUB4,ITAUUNIBANCO,37.49,38.62,37.43,38.01,14074800,53779375200,37.580,36.855714
7843,2017-02-06,ITUB4,ITAUUNIBANCO,38.30,38.69,37.96,38.75,8649200,33053192500,37.734,36.971429
...,...,...,...,...,...,...,...,...,...,...,...
530479,2021-12-22,ITUB4,ITAUUNIBANCO,21.23,21.31,20.97,21.34,18091300,38276646500,21.376,22.265714
530480,2021-12-23,ITUB4,ITAUUNIBANCO,21.33,21.57,21.17,21.56,22377000,47914965700,21.258,22.184762
530481,2021-12-27,ITUB4,ITAUUNIBANCO,21.48,21.70,21.31,21.48,14025800,30256889300,21.280,22.104286
530482,2021-12-28,ITUB4,ITAUUNIBANCO,21.63,21.70,21.40,21.30,15294200,32904978000,21.352,22.040952


In [121]:
qtd_linhas = len(df_itau)

# Definindo os pontos de corte
fim_treino = round(.70 * qtd_linhas)
fim_teste = round(.94 * qtd_linhas) # 70% + 24%

info = (
    f"Linhas Treino: 0 : {fim_treino}\n"
    f"Linhas Teste: {fim_treino} : {fim_teste}\n"
    f"Linhas Validação:  {fim_teste} : {qtd_linhas}"
)
print(info)

Linhas Treino: 0 : 850
Linhas Teste: 850 : 1141
Linhas Validação:  1141 : 1214


In [122]:
#separando as features e labels
features = df_itau.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fechamento'], axis=1)
labels = df_itau['preco_fechamento']

In [123]:
#Escolhendo as melhores features com Kbest

features_list = ('preco_abertura','preco_max','preco_min','qtd_negocios','volume_negocios','mm5d','mm21d')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)


Melhores features:
{'qtd_negocios': np.float64(61.402990891210905), 'preco_min': np.float64(60.67204424327277), 'preco_max': np.float64(54.64691762894059), 'mm21d': np.float64(40.02430332807629), 'volume_negocios': np.float64(1.826497558052192), 'mm5d': np.float64(1.1448774801410078)}
